## 代辦事項：

In [ ]:
# 導入模塊
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np 
import sqlite3
import sys
import datetime
import math
sys.path.append('..')
# silence warnings
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
# sns.set(font='Taipei Sans TC Beta')

# 自定義模塊
import util.swingTrade as st

In [ ]:
pd.set_option('display.max_columns', 50)
# pd.set_option('display.max_rows', 150)

# 要導入的資料

In [ ]:
df = st.getFullData(startDate='2015-01-01', columns=['成交值(千元)', '流通在外股數(千股)'])

In [ ]:
mylist = []
for chunk in  pd.read_csv(f'{st.getRootPath()}/大型資料/前15大買賣超.csv', chunksize=200000):
    mylist.append(chunk)
df_15 = pd.concat(mylist, axis= 0)
del mylist

In [ ]:
df_15 =  df_15.sort_index(level=1).sort_values(by=['年月日', '證券代碼', '前排序'])                                        
df_15 = df_15.set_index(['年月日', '證券代碼'])
df_15

In [ ]:
df_15 = df_15[df_15['券商名稱'] == '台灣摩根士丹利']
df_15

# 大戶分點
凱基-台北
統一-城中
凱基-城中
富邦-嘉義
元大-土城永寧
永豐金-信義
凱基-信義
國票-安和
凱基-市政
統一-南京
群益金鼎-台北
群益金鼎
元富-延平
凱基-大安

# 處理未導入條件

## 價量

In [ ]:

try:
    df['K棒收點'] = (df['收盤價(元)'] - df['最低價(元)']) / (df['最高價(元)'] - df['最低價(元)'])
    df['開盤價(元)'] = df['開盤價(元)'][df['開盤價(元)'] != 'N.A.']
    df['本益比-TEJ'] = df['本益比-TEJ'][df['本益比-TEJ'] != 'N.A.']
    df['開盤價(元)'] = pd.to_numeric(df['開盤價(元)'])
    df['本益比-TEJ'] = pd.to_numeric(df['本益比-TEJ'])
    df['年月日'] = pd.to_datetime(df['年月日'] )
    df['當沖收益'] = ((df['收盤價(元)'] - df['開盤價(元)']) / df['收盤價(元)'])
    df = df.set_index(['年月日', '證券代碼'])
    df['漲跌幅'] = df.groupby('證券代碼')['收盤價(元)'].pct_change() * 100
    df['開收比'] = (df['收盤價(元)'] - df['開盤價(元)']) / df['開盤價(元)'] * 100
    df['成交值等級'] = df.groupby('年月日')['成交值(千元)'].rank(ascending=False)
    df = df[~df.index.duplicated()]
except Exception as e :
    print(e)

df

In [ ]:
pd.merge(df, df_15, on=['年月日', '證券代碼'], how='left')

In [ ]:
trade_position = df.pivot_table(index='年月日', columns='證券代碼', values='開盤價(元)')
trade_position = df.pivot_table(index='年月日', columns='證券代碼', values='收盤價(元)')

In [ ]:
%%time
# 10張以下占比變化	400以上占比變化	
df['放在最後用'] = 1
strategy_map = {
  '進場': (
    # (df.index.get_level_values(1) == '3035 智原') &
    (df.groupby('證券代碼')['是否有波段驅使之融資'].shift(1) == 1) &
    (df.groupby('證券代碼')['融資是否落底'].shift(1) == 1) &
    # ((df['開盤價(元)'] - df.groupby('證券代碼')['收盤價(元)'].shift(6)) / df.groupby('證券代碼')['收盤價(元)'].shift(6) < 0.1 ) &
    # (df['成交值(千元)'] >= 600000) &
    (df['放在最後用'] == 1)
  ),
  '出場': (
    (df.groupby('證券代碼')['是否有波段驅使之融資'].shift(4) == 1) &
    (df.groupby('證券代碼')['融資是否落底'].shift(4) == 1) &
    # (df.groupby('證券代碼')['融資是否回升'].shift(1) == 1) or
    # (df.groupby('證券代碼')['是否有波段驅使之融資'].shift(10) == 1 & df.groupby('證券代碼')['融資是否落底'].shift(10) == 1) |
    # ((df['收盤價(元)'] - df.groupby('證券代碼')['收盤價(元)'].shift(5)) / df.groupby('證券代碼')['收盤價(元)'].shift(5) < -0.1 ) &
    (df['放在最後用'] == 1)
  )
}

In [ ]:
%%time
import importlib
%load_ext autoreload
%autoreload 2
importlib.reload(st)

backtest = st.Backtest(df[['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)']], trade_position=trade_position,  stockCash=500000)
report = backtest.run(strategy_map)
# profit = backtest.run(strategy_map)
# report = backtest.getDetailInfo(strategy_map)
profit = backtest.profit

In [ ]:
%%time
df_stock = backtest.read_stock()
df_stock

In [ ]:
# %%time

# df['放在最後用'] = 1
# strategy_map = {
#   '進場': (
#     (df.index.get_level_values(1) == '3035 智原') &
#     (df.groupby('證券代碼')['是否有波段驅使之融資'].shift(1) == 1) &
#     (df.groupby('證券代碼')['融資是否落底'].shift(1) == 1) &
#     (df['放在最後用'] == 1)
#   ),
#   '出場': (
#     (df.groupby('證券代碼')['是否有波段驅使之融資'].shift(6) == 1) &
#     (df.groupby('證券代碼')['融資是否落底'].shift(6) == 1) &
#     (df['放在最後用'] == 1)
#   )
# }

In [ ]:
backtest.df_position

In [ ]:
%%time
import importlib
%load_ext autoreload
%autoreload 2
importlib.reload(st)

backtest = st.Backtest(df[['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)']], trade_position=trade_position,  stockCash=500000)
report = backtest.run(strategy_map)
# profit = backtest.run(strategy_map)
# report = backtest.getDetailInfo(strategy_map)
profit = backtest.profit

In [ ]:
%%time
df_stock = backtest.read_stock()
df_stock